In [1]:
import os
os.chdir('..')

In [2]:
from src.api.workflows.FAQs.extract_FAQs import obtener_faqs, obtener_faqs_unir
faqs = obtener_faqs(); faqs_unir = obtener_faqs_unir()

In [4]:
from src.api.common.dependency_container import DependencyContainer
from urllib.parse import quote  

DC = DependencyContainer(); DC.initialize()
pregunta = 'perrito, gatico, lobo'
request = {'pregunta': pregunta, 'FAQs': faqs + faqs_unir}
response_str = DependencyContainer.get_faqs_workflow().execute(request)
import json; response = json.loads(response_str)
faq = max(response['FAQs'], key=lambda x: float(x['Score']))  
if float(faq['Score']) >= 6: content = f"<h2>{faq['FAQ']}</h2>{faq['Contenido']}"  
else: content = f'<h2>No encontramos ninguna FAQ relacionada a la pregunta. Dirígete al siguiente número de Whatsapp: <a href="https://web.whatsapp.com/send?phone=34689909323&text={quote(pregunta)}">contacta con un asesor.</a></h2>'
response['FAQs']

[{'FAQ': '8. Acabar todas las actividades para pasar de módulo',
  'Contenido': '<p class="centrar"><img alt="Acabar todo" class="responsive" src="https://becat.online/FAQ/img/acabar-todo.png"/></p><p>Es frecuente que los participantes piensen que han acabado un módulo, cuando en realidad hay alguna actividad que se se ha marcado. El paso de un móculo a otro es automático cuando están marcadas <strong>todas las actividades</strong>. Si falta una, no se considera terminado y no se puede pasar al siguiente módulo. </p><p>En caso de que todas las actividades se encuentren marcadas y el módulo siguiente no se abra, sigue las indicaciones mostradas en <a href="https://becat.online/FAQ/index.html#3"> "3. He acabado el módulo y no me permite pasar al siguiente"</a>.</p>',
  'Razonamiento': 'Esta FAQ es relevante porque aborda directamente el problema de no poder avanzar al siguiente módulo, lo cual puede estar relacionado con la finalización de actividades.',
  'Score': 8},
 {'FAQ': '15. No p